In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    ConfusionMatrixDisplay
)
from matplotlib import pyplot as plt
import os
import torch

In [ ]:
from sklearn.metrics import classification_report

def fbeta_score(precision, recall, beta):
    beta_squared = beta ** 2
    numerator = (1 + beta_squared) * precision * recall
    denominator = beta_squared * precision + recall
    fbeta = numerator / denominator
    return fbeta

def classification_report_fbeta(y_true, y_pred, beta=0.5, target_names=['0', '1', '2']):
    report = classification_report(y_true, y_pred, target_names=target_names, output_dict=True)
    
    fbeta_scores = {}
    for class_name, metrics in report.items():
        if class_name == 'accuracy':
            continue
        precision = metrics['precision']
        recall = metrics['recall']
        fbeta = fbeta_score(precision, recall, beta)
        fbeta_scores[class_name] = fbeta
    
    weighted_precision = report['weighted avg']['precision']
    weighted_recall = report['weighted avg']['recall']
    weighted_fbeta = fbeta_score(weighted_precision, weighted_recall, beta)

    # Print the F0.5 score for each class
    print("F0.5 scores per class:")
    for class_name, fbeta in fbeta_scores.items():
        print(f"{class_name}: {fbeta:.2f}")
    
    return report, weighted_fbeta, fbeta_scores

y_true = [0, 1, 0, 1, 1, 2]
y_pred = [0, 0, 1, 1, 1, 2]
report, weighted_fbeta, fbeta_scores = classification_report_fbeta(y_true, y_pred, beta=0.5)


In [ ]:
def convert_score_to_color(score, YELLOW_GREEN=75/50, RED_YELLOW=30/50):
    LABEL2ID = {"GREEN": 0, "YELLOW": 1, "RED":2}
    red_index = score < RED_YELLOW
    yellow_index = ((score >= RED_YELLOW).int() & (score < YELLOW_GREEN).int()).bool()
    green_index = score >= YELLOW_GREEN

    score[red_index] = LABEL2ID["RED"]
    score[yellow_index] = LABEL2ID["YELLOW"]
    score[green_index] = LABEL2ID["GREEN"]

    return score

def load_pred_and_label(pred_path, label_path):
    pred = np.load(pred_path)
    label = np.load(label_path)

    pred = np.concatenate(pred)
    label = np.concatenate(label)
    index = label != -1    
    print(index.shape, pred.shape, label.shape)
    
    return label[index], pred[index]
def cal_f1(pred_path, label_path, YELLOW_GREEN=80/50, RED_YELLOW=30/50):
    label, pred = load_pred_and_label(pred_path=pred_path, label_path=label_path)
    
    actual = convert_score_to_color(torch.from_numpy(label), YELLOW_GREEN=YELLOW_GREEN, RED_YELLOW=RED_YELLOW)
    predicted = convert_score_to_color(torch.from_numpy(pred), YELLOW_GREEN=YELLOW_GREEN, RED_YELLOW=RED_YELLOW)
    
    f_beta = classification_report_fbeta(y_true=actual, y_pred=predicted, beta=0.5, target_names=["GREEN", "YELLOW", "RED"])
    print(f_beta)
    result = classification_report(y_true=actual, y_pred=predicted)
    cfs_mtr = confusion_matrix(actual, predicted)
    print(result)
    
    cfs_mtr = cfs_mtr / cfs_mtr.sum(axis=1, keepdims=True)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cfs_mtr, display_labels = ["GREEN", "YELLOW", "RED"])
    

    return result, cm_display.plot(cmap='Blues')
    
pred_path = "/data/codes/apa/train/exps/test-short-v2/ckpts-eph=35-mse=0.15209999680519104/phn_pred.npy"
label_path = "/data/codes/apa/train/exps/test-short-v2/ckpts-eph=35-mse=0.15209999680519104/phn_label.npy"

res, cm_display = cal_f1(pred_path=pred_path, label_path=label_path, YELLOW_GREEN=80/50, RED_YELLOW=30/50)
plt.title("Short sentence")
plt.show() 

In [ ]:
def convert_score_to_color(score, YELLOW_GREEN=75/50, RED_YELLOW=30/50):
    LABEL2ID = {"GREEN": 0, "YELLOW": 1, "RED":1}
    red_index = score < RED_YELLOW
    yellow_index = ((score >= RED_YELLOW).int() & (score < YELLOW_GREEN).int()).bool()
    green_index = score >= YELLOW_GREEN

    score[red_index] = LABEL2ID["RED"]
    score[yellow_index] = LABEL2ID["YELLOW"]
    score[green_index] = LABEL2ID["GREEN"]

    return score

def load_pred_and_label(pred_path, label_path):
    pred = np.load(pred_path)
    label = np.load(label_path)

    pred = np.concatenate(pred)
    label = np.concatenate(label)
    index = label != -1    
    print(index.shape, pred.shape, label.shape)
    
    return label[index], pred[index]
def cal_f1(pred_path, label_path, YELLOW_GREEN=70/50, RED_YELLOW=35/50):
    label, pred = load_pred_and_label(pred_path=pred_path, label_path=label_path)
    
    actual = convert_score_to_color(torch.from_numpy(label), YELLOW_GREEN=YELLOW_GREEN, RED_YELLOW=RED_YELLOW)
    predicted = convert_score_to_color(torch.from_numpy(pred), YELLOW_GREEN=YELLOW_GREEN, RED_YELLOW=RED_YELLOW)
    
    f_beta = classification_report_fbeta(y_true=actual, y_pred=predicted, beta=0.5, target_names=["GREEN", "YELLOW"])
    print(f_beta)
    result = classification_report(y_true=actual, y_pred=predicted)
    cfs_mtr = confusion_matrix(actual, predicted)
    print(result)
    
    cfs_mtr = cfs_mtr / cfs_mtr.sum(axis=1, keepdims=True)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cfs_mtr, display_labels = ["GREEN", "YELLOW"])
    

    return result, cm_display.plot(cmap='Blues')
    
pred_path = "/data/codes/apa/train/exps/test-short-v2/ckpts-eph=35-mse=0.15209999680519104/phn_pred.npy"
label_path = "/data/codes/apa/train/exps/test-short-v2/ckpts-eph=35-mse=0.15209999680519104/phn_label.npy"

res, cm_display = cal_f1(pred_path=pred_path, label_path=label_path, YELLOW_GREEN=80/50, RED_YELLOW=30/50)
plt.title("Short sentence")
plt.show() 

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

def load_pred_and_label(pred_path, label_path):
    pred = np.load(pred_path)
    label = np.load(label_path)

    pred = np.concatenate(pred)
    label = np.concatenate(label)
    index = label != -1    
    print(index.shape, pred.shape, label.shape)
    
    return label[index], pred[index]

def cal_utt_acc(pred, label, threshold=10/50):
    _temp = np.abs(pred-label)
    
    pred = _temp < threshold
    label = np.ones_like(_temp)

    print((pred == label).sum()/ label.shape)
    
    print(classification_report(y_true=label, y_pred=pred, zero_division=0))

pred_path = "/data/codes/apa/train/exps/test-short-v2/ckpts-eph=35-mse=0.15209999680519104/utt_pred.npy"
label_path = "/data/codes/apa/train/exps/test-short-v2/ckpts-eph=35-mse=0.15209999680519104/utt_label.npy"
label, pred = load_pred_and_label(pred_path=pred_path, label_path=label_path)
cal_utt_acc(pred=pred, label=label)